# Selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import sys
import csv
import time

In [5]:
# import the webdriver
chrome_options = Options()
chrome_options.headless = False
chrome_options.add_argument("start-maximized")
#chrome_options.addArguments("disable-infobars");



In [131]:
# default path to file to store data
path_to_file = "./Devpost.csv"
# open the file to save the review
csvFile = open(path_to_file, 'w', encoding="utf-8")
fieldnames = ['Hackathon Name', 'Number of participants', 'Host', 'Online/Physical','prizes','interest']
writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
writer.writeheader()

csvWriter = csv.writer(csvFile)

# Scraping

In [132]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
url = "https://devpost.com/hackathons?search=singapore"
browser.get(url)

last_height = browser.execute_script("return document.body.scrollHeight")
print('l',last_height)

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    # Wait to load page
    time.sleep(5)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
        

container = browser.find_elements(By.XPATH, "//div[contains(@class, 'hackathon-tile')]")
store = []
    
for j in range(len(container)):

    name = container[j].find_element(By.XPATH, ".//h3[contains(@class, 'mb-4')]").text
    no_of_participants = int(container[j].find_element(By.XPATH, ".//strong[contains(@class, '')]").text)
    try:
        host = container[j].find_element(By.XPATH, ".//span[@class='label round host-label']").text.replace("\n", "  ")
    except:
        host = ''
    o_p = container[j].find_element(By.XPATH, ".//div[@class='info']").text.replace("\n", "  ")
    prizes = container[j].find_element(By.XPATH, ".//span[@class='prize-amount']").text.replace("\n", "  ")
    print(name, no_of_participants, host,o_p,prizes)
    
    #interests_tags = container[j].find_element(By.XPATH, ".//span[@class='label theme-label mr-2 mb-2']").text
    #print(interests_tags)
    try:
        interest = []
        interests_tags = container[j].find_elements(By.XPATH, ".//span[contains(@class,'label theme-label mr-2 mb-2')]")
        #print('it',interests_tags[0].text)
        for i in range(len(interests_tags)):
            interest.append(interests_tags[i].text)
            #print(interests_tags[i].text)
    except:
        interest = 'None'

    temp_dict = {}
    temp_dict['hackathon'] = name
    temp_dict['participants'] = no_of_participants
    temp_dict['host'] = host
    temp_dict['o_p'] = o_p
    temp_dict['prizes'] = prizes
    temp_dict['interests'] = interest

    store.append(temp_dict)


    csvWriter.writerow([name, no_of_participants, host, o_p,prizes,interest])
    csvFile.flush()

#print("Scraped page " + str(i) + " of " + str(num_page)) 

# Go to next page            
#browser.find_element(By.XPATH, './/a[@class="ui_button nav next primary "]').click()

print(store)
browser.quit()
csvFile.close()

l 2940
TPCC DMIG: The Green Micro:bit Challenge 27 Tanjong Pagar Community Club DMIG Tanjong Pagar Community Club $1,200
Singapore Blockchain Innovation Challenge 2022 5 Singapore Blockchain Innovation Programe Online $10,000
BlockHack Global FE 2022 35 Bitcoin Bay Sheldon & Tracy Levy SLC $CAD400
ADES Project Showcase AY2223S1 65 Singapore Polytechnic, SoC ADES Online $0
SMU LIT Hackathon 2021 103 Singapore Management University Online $3,000
SMU LIT Hackathon 2022 70 Singapore Management University Online $11,500
COVID-19 Idea Sprint with GovTech Singapore 320 Singapore Government Online $0
AWS Hackdays | Special Edition Machine Learning 18 Amazon Singapore $25,000
Deprecated 13 unknown Singapore $1
BlockChamps 2022 132 Ethlas Virtual / Ethlas Office $3,180
Hack For Good 2020 75  Seminar Room 1, COM1 02-06 $0
Accounting Innovation Challenge 2019 133  IMDA PIXEL Studios $10,000
Accounting Innovation Challenge 2018 75  IMDA PIXEL Studios $10,000
GIIS HACKATHON 2022 106 GIIS GIIS Smart 

In [89]:
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200')

In [136]:
es.indices.delete(index='devpost', ignore=[400,404])
es.indices.create(index='devpost', body={
    "mappings":{
        "properties":{
            "hackathon":{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                    }
                }
            },
            "participants":{
                "type":"integer"
            },
            "hosts":{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                    }
                }
            },
            "o_p":{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                    }
                }
            },
            "prizes":{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                    }
                }
            },
            "interests":{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                    }
                }
            },
            
        }
    }
})

# insert keywordbags into elasticsearch
elastic_id = 1
for i in store:
    print(i['hackathon'])
    es.index(index='devpost', id=elastic_id, body=i)
    elastic_id += 1

/var/folders/2n/yj7152_111l7b3hbblxv9pjc0000gn/T/ipykernel_57996/3941967099.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='devpost', ignore=[400,404])
/var/folders/2n/yj7152_111l7b3hbblxv9pjc0000gn/T/ipykernel_57996/3941967099.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index='devpost', ignore=[400,404])
/var/folders/2n/yj7152_111l7b3hbblxv9pjc0000gn/T/ipykernel_57996/3941967099.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='devpost', body={
/var/folders/2n/yj7152_111l7b3hbblxv9pjc0000gn/T/ipykernel_57996/3941967099.py:2: Elastics

TPCC DMIG: The Green Micro:bit Challenge


/var/folders/2n/yj7152_111l7b3hbblxv9pjc0000gn/T/ipykernel_57996/3941967099.py:57: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='devpost', id=elastic_id, body=i)


Singapore Blockchain Innovation Challenge 2022
BlockHack Global FE 2022
ADES Project Showcase AY2223S1
SMU LIT Hackathon 2021
SMU LIT Hackathon 2022
COVID-19 Idea Sprint with GovTech Singapore
AWS Hackdays | Special Edition Machine Learning
Deprecated
BlockChamps 2022
Hack For Good 2020
Accounting Innovation Challenge 2019
Accounting Innovation Challenge 2018
GIIS HACKATHON 2022
Park of the Future Hackathon: Reimagining the Commercial Park
Salt Category Submission
Science 2025: Towards Science Communication for SG60
YouthHacks
What The Hack 2020: Environment Edition
COMIT Hackathon - Atomic Swaps for your Application
DigiFest
#OMRS15 | Mingle, Learn, Share and Build reality.
FOSSASIA UNESCO Open Science and Data Hackathon
CodeCircles Data Hackathon
GIIS Hackathon 2k18
CodeFiesta 2021 Final Submission
AWS Build On 2022
SG100 Nation Hackathon
#codeathonX KL 2019
KVision x VIISA Hackathon: Building Co-Innovation
Knowlarity API Hackathon
IDEATE 2021
#codeForCorona: Students Against COVID
i

# Topic Modelling

In [60]:
# Importing modules
import pandas as pd
import os

# Read data
data = pd.read_csv('./devpost.csv', encoding='utf-8')

data.head()

,Hackathon Name,Number of participants,Host,Online/Physical,prizes
0,TPCC DMIG: The Green Micro:bit Challenge,22,Tanjong Pagar Community Club DMIG,Tanjong Pagar Community Club,"$1,200"
1,BlockHack Global FE 2022,25,Bitcoin Bay,Sheldon & Tracy Levy SLC,$CAD400
2,ADES Project Showcase AY2223S1,65,"Singapore Polytechnic, SoC ADES",Online,$0
3,SMU LIT Hackathon 2021,103,Singapore Management University,Online,"$3,000"
4,SMU LIT Hackathon 2022,70,Singapore Management University,Online,"$11,500"


In [ ]:
# Remove the columns
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1)

# Print out the first rows of papers
papers.head()